In [5]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
import numpy as np
import pandas as pd
import glob
import csv
import datetime as dt
#import geopy
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker
import xlrd
import math
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [6]:
def haversine(coord1, coord2):
    R =  3958.8 # Earth radius in miles //in meters 6372800
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))


In [11]:
               
                
data = csv.reader(open('TrafficDataReduced_v3.csv'))
nb_data_cell=[]
sb_data_cell=[]
all_data_cell=[]
col = True
for i in data:
    #cell = []
    if col == False:

        #starttime = dt.datetime.strptime('6/27/2018 00:00:00', '%m/%d/%Y %H:%M:%S')
        #T2 = dt.datetime.strptime(z[3], '%Y-%m-%d %H:%M:%S.%f')
        #T1 = T2 - dt.timedelta(minutes=60)
        #T3 = T2 + dt.timedelta(minutes=60)
        cell=[]
        starttime = dt.datetime.strptime('6/27/2018 00:00:00', '%m/%d/%Y %H:%M:%S')
        #T2 = dt.datetime.strptime(z[3], '%Y-%m-%d %H:%M:%S.%f')
        #T1 = T2 - dt.timedelta(minutes=60)
        #T3 = T2 + dt.timedelta(minutes=60)
        timecell=dt.datetime.strptime(i[4]+' '+i[5], '%m/%d/%Y %H:%M:%S') #.timestamp()
        seconds=(timecell - starttime).total_seconds()/60

        i.insert(12, seconds)
        #cell.append(i[6])
        if i[8].strip()=='nb':
            dist_lat, dist_lon = 32.958786, -83.813435 # standard point of NB (at the beginning of NB road)
        elif i[8].strip()=='sb':
            dist_lat, dist_lon = 32.753620, -83.711824
        cell_lon = float(i[2])
        cell_lat = float(i[1])

        lat1 = dist_lat; lon1=dist_lon; lat2=cell_lat; lon2=cell_lon


        coords_1 = (dist_lat, dist_lon)
        coords_2 = (cell_lat, cell_lon)
        distance = haversine(coords_1, coords_2)*5
        i.insert(13,-distance)   

        if i[8].strip()=='nb': # and i[4]=='7/21/2018'
            nb_data_cell.append(i)
        elif i[8].strip()=='sb':
            sb_data_cell.append(i)

        all_data_cell.append(i)
    else:
        col_name = i
        col=False
        
pd.DataFrame(all_data_cell).to_csv('TrafficDataReduced_v4.csv', header=None, index=None)
#csv.writer(open('myfile.csv','w')).writerows(all_data_cell)
        
XXX = {'NB':np.array(nb_data_cell), 'SB':np.array(sb_data_cell)}


Since density-based clustering has a single epsilon with minumum points 

In [9]:

eps_value= 15*15
minSamples = 15

N_table_core = np.zeros((eps_value,minSamples))
S_table_core = np.zeros((eps_value,minSamples))
N_table_cluster = N_table_core
S_table_cluster = S_table_core


for i in np.arange(1,minSamples+1): #eps_time
    for ii in np.arange(1,minSamples+1):#eps_distance
        for z in range(1,minSamples+1): #min_samples
            for key, X in XXX.items():
                XX=X[:,12:14].astype(np.float)
                XX=XX//np.array([i,ii])
                
                n_clusters = 0
                
                db = DBSCAN(eps=1, min_samples=z).fit(XX)
                core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
                core_samples_mask[db.core_sample_indices_] = True
                labels = db.labels_
                
                
                # Number of clusters in labels, ignoring noise if present.
                n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
                n_samples = X.shape[0]
                n_core_samples = list(core_samples_mask).count(True)
                n_noise_ = list(labels).count(-1)
                n_border_samples = n_samples - n_noise_ - n_core_samples
                
                
                '''
                print('Estimated number of samples: %d' % n_samples)
                print('Estimated number of clusters: %d' % n_clusters_)
                print('Estimated number of core samples: %d' % n_core_samples)
                print('Estimated number of border samples: %d' % n_border_samples)
                print('Estimated number of noise points: %d' % n_noise_)
                print("Silhouette Coefficient: %0.3f"
                      % metrics.silhouette_score(X, labels))
                import matplotlib.pyplot as plt
                
                # Black removed and is used for noise instead.
                unique_labels = set(labels)
                colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
                for k, col in zip(unique_labels, colors):
                    if k == -1:
                        # Black used for noise.
                        col = [0, 0, 0, 1]
                    
                    class_member_mask = (labels == k)
                    
                    xy = X[class_member_mask & core_samples_mask]
                    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                             markeredgecolor='k', markersize=10)
                    
                    xy = X[class_member_mask & ~core_samples_mask]
                    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                             markeredgecolor='k', markersize=1)
                plt.title(key+', Estimated number of clusters:' +str(n_clusters_)+ ', eps:' +str(i)+ ', min_samples:' +str(z))
                plt.savefig('./'+str(key)+'_'+str(i)+'_'+str(z)+'.png', dpi=1000)
                #print(key, i, z)
                plt.close()
                '''
    
                print(key,15*(i-1)+ii,z,n_clusters_,n_core_samples)
    
                #'''
                if key == 'NB':
                    N_table_core[15*(i-1)+ii-1][z-1]=n_core_samples
                    N_table_cluster[15*(i-1)+ii-1][z-1]=n_clusters_
                else:
                    S_table_core[15*(i-1)+ii-1][z-1]=n_core_samples
                    S_table_cluster[15*(i-1)+ii-1][z-1]=n_clusters_
                #print (key,i,z)
                #'''
#np.savetxt('nb_table_'++'.csv', N_table_core, delimiter=",")
#np.savetxt('sb_table_'++'.csv', S_table_core, delimiter=",")
#np.savetxt('nb_table_cluster_721.csv', N_table_cluster, delimiter=",", fmt='% 4d')
#np.savetxt('sb_table_cluster_721.csv', S_table_cluster, delimiter=",", fmt='% 4d')
        


NB 1 1 4301 4757
SB 1 1 4253 4658
NB 1 2 400 856
SB 1 2 369 774
NB 1 3 43 130
SB 1 3 33 93
NB 1 4 9 33
SB 1 4 2 5
NB 1 5 2 7
SB 1 5 0 0
NB 1 6 1 2
SB 1 6 0 0
NB 1 7 0 0
SB 1 7 0 0
NB 1 8 0 0
SB 1 8 0 0
NB 1 9 0 0
SB 1 9 0 0
NB 1 10 0 0
SB 1 10 0 0
NB 1 11 0 0
SB 1 11 0 0
NB 1 12 0 0
SB 1 12 0 0
NB 1 13 0 0
SB 1 13 0 0
NB 1 14 0 0
SB 1 14 0 0
NB 1 15 0 0
SB 1 15 0 0
NB 2 1 4259 4757
SB 2 1 4191 4658
NB 2 2 425 923
SB 2 2 423 890
NB 2 3 50 151
SB 2 3 37 109
NB 2 4 11 41
SB 2 4 4 9
NB 2 5 2 10
SB 2 5 0 0
NB 2 6 1 5
SB 2 6 0 0
NB 2 7 0 0
SB 2 7 0 0
NB 2 8 0 0
SB 2 8 0 0
NB 2 9 0 0
SB 2 9 0 0
NB 2 10 0 0
SB 2 10 0 0
NB 2 11 0 0
SB 2 11 0 0
NB 2 12 0 0
SB 2 12 0 0
NB 2 13 0 0
SB 2 13 0 0
NB 2 14 0 0
SB 2 14 0 0
NB 2 15 0 0
SB 2 15 0 0
NB 3 1 4248 4757
SB 3 1 4173 4658
NB 3 2 433 942
SB 3 2 434 919
NB 3 3 50 158
SB 3 3 38 119
NB 3 4 11 46
SB 3 4 5 17
NB 3 5 3 11
SB 3 5 1 6
NB 3 6 1 5
SB 3 6 1 3
NB 3 7 0 0
SB 3 7 0 0
NB 3 8 0 0
SB 3 8 0 0
NB 3 9 0 0
SB 3 9 0 0
NB 3 10 0 0
SB 3 10 0 0
NB 3 11 0

SB 21 14 0 0
NB 21 15 0 0
SB 21 15 0 0
NB 22 1 4099 4757
SB 22 1 4056 4658
NB 22 2 484 1142
SB 22 2 488 1090
NB 22 3 77 304
SB 22 3 64 221
NB 22 4 29 155
SB 22 4 16 80
NB 22 5 18 80
SB 22 5 9 41
NB 22 6 7 29
SB 22 6 5 22
NB 22 7 4 19
SB 22 7 5 19
NB 22 8 2 8
SB 22 8 1 11
NB 22 9 1 3
SB 22 9 1 10
NB 22 10 0 0
SB 22 10 1 7
NB 22 11 0 0
SB 22 11 1 5
NB 22 12 0 0
SB 22 12 0 0
NB 22 13 0 0
SB 22 13 0 0
NB 22 14 0 0
SB 22 14 0 0
NB 22 15 0 0
SB 22 15 0 0
NB 23 1 4085 4757
SB 23 1 4038 4658
NB 23 2 496 1168
SB 23 2 499 1119
NB 23 3 81 317
SB 23 3 69 240
NB 23 4 29 151
SB 23 4 15 75
NB 23 5 17 81
SB 23 5 11 45
NB 23 6 7 26
SB 23 6 3 18
NB 23 7 5 18
SB 23 7 3 15
NB 23 8 2 11
SB 23 8 1 12
NB 23 9 0 0
SB 23 9 1 12
NB 23 10 0 0
SB 23 10 1 12
NB 23 11 0 0
SB 23 11 1 8
NB 23 12 0 0
SB 23 12 1 6
NB 23 13 0 0
SB 23 13 0 0
NB 23 14 0 0
SB 23 14 0 0
NB 23 15 0 0
SB 23 15 0 0
NB 24 1 4078 4757
SB 24 1 4026 4658
NB 24 2 498 1177
SB 24 2 504 1136
NB 24 3 80 316
SB 24 3 67 241
NB 24 4 28 162
SB 24 4 15 82
N

SB 41 7 6 40
NB 41 8 5 49
SB 41 8 4 24
NB 41 9 3 38
SB 41 9 3 18
NB 41 10 2 26
SB 41 10 2 13
NB 41 11 2 24
SB 41 11 1 9
NB 41 12 3 11
SB 41 12 1 9
NB 41 13 0 0
SB 41 13 1 4
NB 41 14 0 0
SB 41 14 0 0
NB 41 15 0 0
SB 41 15 0 0
NB 42 1 3920 4757
SB 42 1 3913 4658
NB 42 2 572 1409
SB 42 2 559 1304
NB 42 3 103 433
SB 42 3 96 348
NB 42 4 39 237
SB 42 4 21 125
NB 42 5 24 167
SB 42 5 12 79
NB 42 6 15 117
SB 42 6 6 54
NB 42 7 9 77
SB 42 7 6 45
NB 42 8 6 52
SB 42 8 4 28
NB 42 9 2 37
SB 42 9 3 19
NB 42 10 2 26
SB 42 10 2 14
NB 42 11 2 24
SB 42 11 1 9
NB 42 12 3 11
SB 42 12 1 9
NB 42 13 0 0
SB 42 13 1 4
NB 42 14 0 0
SB 42 14 0 0
NB 42 15 0 0
SB 42 15 0 0
NB 43 1 3893 4757
SB 43 1 3880 4658
NB 43 2 587 1451
SB 43 2 586 1364
NB 43 3 114 458
SB 43 3 102 361
NB 43 4 38 211
SB 43 4 19 122
NB 43 5 24 159
SB 43 5 12 81
NB 43 6 17 113
SB 43 6 7 53
NB 43 7 9 65
SB 43 7 6 47
NB 43 8 8 45
SB 43 8 4 30
NB 43 9 4 29
SB 43 9 3 19
NB 43 10 1 19
SB 43 10 2 15
NB 43 11 2 12
SB 43 11 1 10
NB 43 12 0 0
SB 43 12 1 10

SB 60 9 5 34
NB 60 10 5 47
SB 60 10 2 22
NB 60 11 3 30
SB 60 11 1 14
NB 60 12 1 25
SB 60 12 1 14
NB 60 13 1 23
SB 60 13 1 14
NB 60 14 1 17
SB 60 14 1 12
NB 60 15 1 10
SB 60 15 1 12
NB 61 1 4142 4757
SB 61 1 4124 4658
NB 61 2 469 1084
SB 61 2 444 978
NB 61 3 66 256
SB 61 3 55 178
NB 61 4 26 132
SB 61 4 11 48
NB 61 5 14 86
SB 61 5 6 36
NB 61 6 12 60
SB 61 6 5 28
NB 61 7 7 37
SB 61 7 4 26
NB 61 8 5 21
SB 61 8 3 21
NB 61 9 2 7
SB 61 9 1 9
NB 61 10 2 7
SB 61 10 0 0
NB 61 11 0 0
SB 61 11 0 0
NB 61 12 0 0
SB 61 12 0 0
NB 61 13 0 0
SB 61 13 0 0
NB 61 14 0 0
SB 61 14 0 0
NB 61 15 0 0
SB 61 15 0 0
NB 62 1 4056 4757
SB 62 1 4022 4658
NB 62 2 498 1199
SB 62 2 508 1144
NB 62 3 77 333
SB 62 3 70 228
NB 62 4 29 171
SB 62 4 13 70
NB 62 5 18 116
SB 62 5 5 50
NB 62 6 11 87
SB 62 6 5 39
NB 62 7 8 67
SB 62 7 4 36
NB 62 8 8 47
SB 62 8 3 27
NB 62 9 5 23
SB 62 9 3 19
NB 62 10 5 21
SB 62 10 2 7
NB 62 11 3 13
SB 62 11 1 4
NB 62 12 2 3
SB 62 12 0 0
NB 62 13 0 0
SB 62 13 0 0
NB 62 14 0 0
SB 62 14 0 0
NB 62 15 0 

SB 79 8 3 53
NB 79 9 8 73
SB 79 9 2 46
NB 79 10 5 56
SB 79 10 3 30
NB 79 11 3 43
SB 79 11 2 20
NB 79 12 2 33
SB 79 12 1 14
NB 79 13 2 33
SB 79 13 1 13
NB 79 14 2 33
SB 79 14 1 12
NB 79 15 2 21
SB 79 15 1 12
NB 80 1 3911 4757
SB 80 1 3847 4658
NB 80 2 562 1408
SB 80 2 606 1417
NB 80 3 110 464
SB 80 3 114 363
NB 80 4 35 244
SB 80 4 23 123
NB 80 5 20 187
SB 80 5 9 86
NB 80 6 15 153
SB 80 6 4 65
NB 80 7 12 121
SB 80 7 4 60
NB 80 8 10 110
SB 80 8 3 52
NB 80 9 6 77
SB 80 9 3 46
NB 80 10 6 71
SB 80 10 2 30
NB 80 11 4 54
SB 80 11 2 23
NB 80 12 2 35
SB 80 12 2 23
NB 80 13 3 28
SB 80 13 1 17
NB 80 14 1 21
SB 80 14 1 17
NB 80 15 1 21
SB 80 15 1 11
NB 81 1 3868 4757
SB 81 1 3844 4658
NB 81 2 589 1478
SB 81 2 595 1409
NB 81 3 115 490
SB 81 3 118 405
NB 81 4 37 257
SB 81 4 24 132
NB 81 5 22 189
SB 81 5 15 95
NB 81 6 17 161
SB 81 6 5 66
NB 81 7 12 121
SB 81 7 5 61
NB 81 8 10 111
SB 81 8 5 53
NB 81 9 6 76
SB 81 9 5 48
NB 81 10 6 67
SB 81 10 2 29
NB 81 11 4 49
SB 81 11 2 19
NB 81 12 2 36
SB 81 12 2 19


...

SB 150 10 3 71
NB 150 11 7 113
SB 150 11 3 65
NB 150 12 6 83
SB 150 12 3 62
NB 150 13 5 70
SB 150 13 2 55
NB 150 14 4 62
SB 150 14 2 46
NB 150 15 3 54
SB 150 15 2 37
NB 151 1 4037 4757
SB 151 1 4019 4658
NB 151 2 530 1250
SB 151 2 498 1137
NB 151 3 85 334
SB 151 3 76 254
NB 151 4 32 182
SB 151 4 14 84
NB 151 5 18 128
SB 151 5 7 54
NB 151 6 13 107
SB 151 6 3 46
NB 151 7 11 88
SB 151 7 3 46
NB 151 8 10 71
SB 151 8 4 41
NB 151 9 7 48
SB 151 9 2 30
NB 151 10 5 27
SB 151 10 2 19
NB 151 11 3 15
SB 151 11 2 10
NB 151 12 0 0
SB 151 12 1 1
NB 151 13 0 0
SB 151 13 0 0
NB 151 14 0 0
SB 151 14 0 0
NB 151 15 0 0
SB 151 15 0 0
NB 152 1 3910 4757
SB 152 1 3874 4658
NB 152 2 575 1422
SB 152 2 579 1363
NB 152 3 103 443
SB 152 3 108 357
NB 152 4 33 237
SB 152 4 22 120
NB 152 5 23 178
SB 152 5 6 74
NB 152 6 14 144
SB 152 6 5 60
NB 152 7 11 123
SB 152 7 4 55
NB 152 8 10 118
SB 152 8 4 51
NB 152 9 10 89
SB 152 9 4 51
NB 152 10 7 69
SB 152 10 3 40
NB 152 11 3 47
SB 152 11 2 30
NB 152 12 3 26
SB 152 12 2 30


SB 168 1 3780 4658
NB 168 2 615 1543
SB 168 2 623 1501
NB 168 3 129 517
SB 168 3 129 452
NB 168 4 37 255
SB 168 4 36 168
NB 168 5 24 203
SB 168 5 11 98
NB 168 6 15 172
SB 168 6 7 86
NB 168 7 12 144
SB 168 7 5 68
NB 168 8 9 128
SB 168 8 4 59
NB 168 9 8 119
SB 168 9 3 58
NB 168 10 7 95
SB 168 10 2 51
NB 168 11 6 84
SB 168 11 2 42
NB 168 12 5 68
SB 168 12 2 39
NB 168 13 3 53
SB 168 13 2 38
NB 168 14 2 45
SB 168 14 2 38
NB 168 15 2 45
SB 168 15 2 29
NB 169 1 3759 4757
SB 169 1 3697 4658
NB 169 2 640 1638
SB 169 2 647 1608
NB 169 3 150 599
SB 169 3 157 538
NB 169 4 48 291
SB 169 4 46 194
NB 169 5 21 211
SB 169 5 17 120
NB 169 6 12 166
SB 169 6 8 96
NB 169 7 11 153
SB 169 7 5 77
NB 169 8 9 142
SB 169 8 4 67
NB 169 9 8 135
SB 169 9 3 60
NB 169 10 8 114
SB 169 10 3 60
NB 169 11 8 97
SB 169 11 2 53
NB 169 12 7 87
SB 169 12 2 46
NB 169 13 3 61
SB 169 13 2 46
NB 169 14 2 47
SB 169 14 2 39
NB 169 15 2 43
SB 169 15 2 30
NB 170 1 3685 4757
SB 170 1 3607 4658
NB 170 2 671 1743
SB 170 2 693 1744
NB 17

SB 185 5 22 132
NB 185 6 18 187
SB 185 6 13 102
NB 185 7 15 171
SB 185 7 6 86
NB 185 8 11 154
SB 185 8 4 69
NB 185 9 11 139
SB 185 9 4 64
NB 185 10 9 121
SB 185 10 3 62
NB 185 11 8 106
SB 185 11 3 55
NB 185 12 6 75
SB 185 12 2 53
NB 185 13 4 52
SB 185 13 2 49
NB 185 14 3 39
SB 185 14 2 49
NB 185 15 3 39
SB 185 15 2 41
NB 186 1 3565 4757
SB 186 1 3533 4658
NB 186 2 717 1909
SB 186 2 715 1840
NB 186 3 205 790
SB 186 3 191 706
NB 186 4 68 389
SB 186 4 62 299
NB 186 5 35 266
SB 186 5 25 154
NB 186 6 22 211
SB 186 6 14 112
NB 186 7 15 174
SB 186 7 6 88
NB 186 8 11 155
SB 186 8 5 75
NB 186 9 10 136
SB 186 9 5 75
NB 186 10 9 126
SB 186 10 3 66
NB 186 11 7 101
SB 186 11 3 58
NB 186 12 6 75
SB 186 12 2 49
NB 186 13 3 60
SB 186 13 2 49
NB 186 14 2 53
SB 186 14 2 44
NB 186 15 2 53
SB 186 15 2 44
NB 187 1 3525 4757
SB 187 1 3460 4658
NB 187 2 733 1965
SB 187 2 747 1945
NB 187 3 211 825
SB 187 3 210 760
NB 187 4 68 405
SB 187 4 75 326
NB 187 5 35 290
SB 187 5 28 168
NB 187 6 21 221
SB 187 6 17 124


SB 202 8 4 72
NB 202 9 11 160
SB 202 9 3 71
NB 202 10 9 150
SB 202 10 3 69
NB 202 11 7 135
SB 202 11 3 69
NB 202 12 6 112
SB 202 12 3 66
NB 202 13 5 97
SB 202 13 3 59
NB 202 14 4 83
SB 202 14 2 52
NB 202 15 4 70
SB 202 15 2 36
NB 203 1 3445 4757
SB 203 1 3405 4658
NB 203 2 776 2088
SB 203 2 752 2005
NB 203 3 229 872
SB 203 3 221 815
NB 203 4 74 434
SB 203 4 79 352
NB 203 5 40 293
SB 203 5 33 195
NB 203 6 24 242
SB 203 6 20 142
NB 203 7 17 197
SB 203 7 11 105
NB 203 8 12 180
SB 203 8 7 83
NB 203 9 10 159
SB 203 9 4 77
NB 203 10 9 155
SB 203 10 3 70
NB 203 11 7 134
SB 203 11 3 70
NB 203 12 6 116
SB 203 12 3 68
NB 203 13 5 101
SB 203 13 3 61
NB 203 14 4 86
SB 203 14 3 47
NB 203 15 4 74
SB 203 15 2 42
NB 204 1 3390 4757
SB 204 1 3340 4658
NB 204 2 786 2153
SB 204 2 768 2086
NB 204 3 240 934
SB 204 3 240 876
NB 204 4 89 469
SB 204 4 88 394
NB 204 5 44 308
SB 204 5 41 206
NB 204 6 28 250
SB 204 6 23 148
NB 204 7 17 187
SB 204 7 9 96
NB 204 8 12 173
SB 204 8 5 83
NB 204 9 9 155
SB 204 9 4 79


SB 219 11 4 53
NB 219 12 7 100
SB 219 12 3 48
NB 219 13 5 88
SB 219 13 3 46
NB 219 14 4 80
SB 219 14 2 45
NB 219 15 4 80
SB 219 15 2 36
NB 220 1 3231 4757
SB 220 1 3212 4658
NB 220 2 834 2360
SB 220 2 831 2277
NB 220 3 290 1060
SB 220 3 282 974
NB 220 4 111 543
SB 220 4 94 425
NB 220 5 46 340
SB 220 5 38 227
NB 220 6 29 267
SB 220 6 24 176
NB 220 7 17 208
SB 220 7 17 132
NB 220 8 12 182
SB 220 8 8 95
NB 220 9 11 165
SB 220 9 5 88
NB 220 10 8 155
SB 220 10 4 80
NB 220 11 6 137
SB 220 11 3 76
NB 220 12 6 109
SB 220 12 3 73
NB 220 13 5 96
SB 220 13 3 67
NB 220 14 5 70
SB 220 14 3 61
NB 220 15 4 69
SB 220 15 2 56
NB 221 1 3132 4757
SB 221 1 3121 4658
NB 221 2 870 2495
SB 221 2 844 2381
NB 221 3 318 1169
SB 221 3 298 1087
NB 221 4 122 598
SB 221 4 119 489
NB 221 5 56 365
SB 221 5 49 259
NB 221 6 32 263
SB 221 6 25 193
NB 221 7 19 219
SB 221 7 18 137
NB 221 8 12 179
SB 221 8 8 98
NB 221 9 11 159
SB 221 9 7 89
NB 221 10 11 146
SB 221 10 5 70
NB 221 11 8 117
SB 221 11 4 64
NB 221 12 5 89
SB 22

In [12]:
A=N_table_cluster==12
B=S_table_cluster==3
np.where(A&B)
N_table_cluster[np.where(A&B)]
S_table_cluster[np.where(A&B)]

i,ii=divmod(np.where(A&B)[0],15)
accuracy_table = []
for r in range(len(i)):
    accuracy_cell=[]
    for key, X in XXX.items():
        XX=X[:,12:14].astype(np.float)          
        XX=XX//np.array([i[r]+1,ii[r]+1])
        n_clusters = 0
        
        db = DBSCAN(eps=1, min_samples=np.where(A&B)[1][r]+1).fit(XX)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_
        
        # np.where(labels!=-1) is instances in any clusters
        #
        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_samples = XX.shape[0]
        n_core_samples = list(core_samples_mask).count(True)
        n_noise_ = list(labels).count(-1)
        n_border_samples = n_samples - n_noise_ - n_core_samples
        
        newcell = np.concatenate((X[:,:14], labels[:,np.newaxis]), axis=1)
        eps_time_scale = i[r]+1
        eps_distance_scale = ii[r]+1
        csv_name = str(key)+'_table_'+str(eps_time_scale)+'_'+str(eps_distance_scale)+'.csv'
        pd.DataFrame(newcell).to_csv(csv_name, header=None, index=None)
        #newcell[:,11]=='non-accident'**newcell[:,14]=='-1'
        
        TN=newcell[:,11][newcell[:,14]=='-1']=='non-accident'
        TP=newcell[:,11][newcell[:,14]=='0']=='accident'
        inc,fea=newcell.shape
        accuracy_point = (TN.size+TP.size)/inc
        print(accuracy_point, key, r)
        accuracy_cell.append(key)
        accuracy_cell.append(eps_time_scale)
        accuracy_cell.append(eps_distance_scale)
        accuracy_cell.append(n_samples)
        accuracy_cell.append(n_clusters_)
        accuracy_cell.append(n_core_samples)
        accuracy_cell.append(n_border_samples)
        accuracy_cell.append(n_noise_)
        accuracy_cell.append(accuracy_point)
        
        #np.savetxt(csv_name, newcell, delimiter=",")
        
        print('Estimated number of samples: %d' % n_samples)
        print('Estimated number of clusters: %d' % n_clusters_)
        print('Estimated number of core samples: %d' % n_core_samples)
        print('Estimated number of border samples: %d' % n_border_samples)
        print('Estimated number of noise points: %d' % n_noise_)
        print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(XX, labels))
        import matplotlib.pyplot as plt
        
        # Black removed and is used for noise instead.
        unique_labels = set(labels)
        colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
        for k, col in zip(unique_labels, colors):
            if k == -1:
                # Black used for noise.
                col = [0, 0, 0, 1]
            
            class_member_mask = (labels == k)
            
            xy = X[class_member_mask & core_samples_mask]
            plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                     markeredgecolor='k', markersize=10)
            
            xy = X[class_member_mask & ~core_samples_mask]
            plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                     markeredgecolor='k', markersize=1)
        plt.title(key+', Estimated number of clusters:' +str(n_clusters_)+ ', eps:' +str(i[r]+1)+', '+str(ii[r]+1)+ ', min_samples:' +str(z))
        plt.savefig('./'+str(key)+'_'+str(i[r]+1)+'_'+str(ii[r]+1)+'.png', dpi=1000)
        #print(key, i, z)
        plt.close()
    
        print(key,i[r]+1,ii[r]+1,n_clusters_,n_core_samples)
    accuracy_table.append(accuracy_cell)
    
print(accuracy_table) 
pd.DataFrame(accuracy_table).to_csv('accuracy_table.csv', header=['direction','eps_time_scale','eps_distance_scale','n_samples','n_clusters_','n_core_samples','n_border_samples','n_noise_','accuracy','direction','eps_time_scale','eps_distance_scale','n_samples','n_clusters_','n_core_samples','n_border_samples','n_noise_','accuracy'], index=None)       


0.9829724616354846 NB 0
Estimated number of samples: 4757
Estimated number of clusters: 12
Estimated number of core samples: 54
Estimated number of border samples: 36
Estimated number of noise points: 4667
Silhouette Coefficient: -0.773
NB 4 1 12 54
0.9965650493774152 SB 0
Estimated number of samples: 4658
Estimated number of clusters: 3
Estimated number of core samples: 21
Estimated number of border samples: 4
Estimated number of noise points: 4633
Silhouette Coefficient: -0.195
SB 4 1 3 21
0.9690981711162497 NB 1
Estimated number of samples: 4757
Estimated number of clusters: 12
Estimated number of core samples: 113
Estimated number of border samples: 43
Estimated number of noise points: 4601
Silhouette Coefficient: -0.800
NB 5 9 12 113
0.9886217260626878 SB 1
Estimated number of samples: 4658
Estimated number of clusters: 3
Estimated number of core samples: 54
Estimated number of border samples: 13
Estimated number of noise points: 4591
Silhouette Coefficient: -0.535
SB 5 9 3 54
0.9